# Lab 11 - Redukcja wymiarowości
- PCA
- t-SNE

## PCA (Principal Component Analysis)
Analiza głównych składników jest szybką metodą redukcji wymiarowości danych.
Polega na znalezieniu liniowej transformacji zbioru zmiennych początkowych w mniej liczny zbiór zmiennych, zwanych składowymi głównymi. Jest to pewien sposób kompresji danych, dlatego metodę tę można wykorzystać w wielu dziedzinach nauki.


**Algorytm PCA:**

1. Standaryzacja danych

2. Wyznaczenie macierzy kowariancji $Σ$ między zmiennymi początkowymi
$$
Σ=\left(\begin{array}{cc} 
var(x_1) & cov(x_1,x_2) & ... & cov(x_1,x_n) \\
cov(x_1, x_2) & var(x_2) & ... & cov(x_2,x_n)\\
... & ... &... & ...\\
cov(x_1, x_2) & cov(x_2,x_n) & ... & var(x_n)\\
\end{array}\right)
$$ 

3. Wyznaczenie wartości własnych macierzy kowariancji poprzez rozwiązanie równania charakterystycznego $|Σ -λI|=0$

4. Wybranie $k$ największych wartości własnych $λ$ i wyznaczenie dla nich wektorów własnych $Z_i$ zgodnie ze wzorem: $(Σ-λ_iI) * Z_i=0$

5. Utworzenie macierzy przekształcenia liniowego $W$, bazującej na wektorach własnych

6. Przekształcenie zmiennych początkowych według wzoru $Y = WX$

![](./fig/pca_matrix.png)
![](./fig/pca_gif.gif)

### Przykład dla sztucznych danych

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [ ]:
rng = np.random.RandomState(1)
X = np.dot(rng.rand(2, 2), rng.randn(2, 200)).T
X.shape

In [ ]:
plt.figure(figsize=(8,5))
plt.scatter(X[:, 0], X[:, 1])
plt.show()

#### Jeden komponent

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
pca.fit(X)

In [ ]:
# Kierunki zmiennych PCA
pca.components_

In [ ]:
# Wyjaśniona wariancja - dla celów dydaktycznych (w praktyce raczej analizujemy procent wyjaśnionej wariancji)
print(f'Total variance: {np.sqrt(np.std(X)):.4f}')
pca.explained_variance_

In [ ]:
# Procent wyjaśnionej wariancji
pca.explained_variance_ratio_

In [ ]:
def draw_vector(v0, v1, ax=None):
    ax = ax or plt.gca()
    arrowprops=dict(arrowstyle='->',
                    linewidth=4,
                    shrinkA=0, shrinkB=0, color='black')
    ax.annotate('', v1, v0, arrowprops=arrowprops)

# plot data
plt.figure(figsize=(8,5))
plt.scatter(X[:, 0], X[:, 1])
plt.axis('equal')
for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector(pca.mean_, pca.mean_ + v)
plt.show()

#### 2 komponenty

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(X)

In [ ]:
# Kierunki zmiennych PCA
pca.components_

In [ ]:
# Wyjaśniona wariancja 
print(f'Total variance: {np.sqrt(np.std(X)):.4f}')
pca.explained_variance_

In [ ]:
# Procent wyjaśnionej wariancji
pca.explained_variance_ratio_

In [ ]:
# plot data
plt.figure(figsize=(8,5))
plt.scatter(X[:, 0], X[:, 1])
plt.axis('equal')
for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector(pca.mean_, pca.mean_ + v)
plt.show()

### Dobieranie odpowiedniej liczby komponentów

Major League Baseball Data from the 1986 and 1987 seasons.

In [ ]:
import pandas as pd
hitters = pd.read_csv('hitters.csv')
hitters.head()

#### Wydzielenie zmiennej celu i podział na zbiór treningowy i testowy

In [ ]:
from sklearn.model_selection import train_test_split

y = hitters['NewLeague']
X = hitters.drop(['Name','NewLeague','League','Division'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA().fit(X_train)

plt.figure(figsize=(9,6))
plt.plot(range(1, len(pca.explained_variance_ratio_)+1), np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

#### Zobaczmy jak poradzi sobie klasyfikacja dla surowych danych i dla PCA

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
model = RandomForestClassifier(random_state=0)

# Surowe dane
y_hat = model.fit(X_train,y_train).predict(X_test)
print(f'Accuracy bez PCA: {accuracy_score(y_test, y_hat):.4f}')

# PCA - odczytane z wykresu
comp_acc_pairs = []
for i in range(1,17):
    pca = PCA(n_components=i)
    y_hat = model.fit(pca.fit_transform(X_train),y_train).predict(pca.transform(X_test))
    comp_acc_pairs.append((i, accuracy_score(y_test, y_hat)))
    
plt.figure(figsize=(9,6))
plt.plot(*list(zip(*comp_acc_pairs)),'bx-')
plt.xlabel('number of components')
plt.ylabel('Accuracy');

## T-SNE

Pytanie: Jak zwizualizować dane wielowymiarowe?

**t-distributed Stochastic Neighbor Embedding**

Cel: Pokazać ukryte zależności wielowymiarowych danych w 2D lub 3D.

Idea: Chcemy, aby obserwacje podobne do siebie w wielu wymiarach były blisko siebie w podprzestrzeni.

Wyróżnijmy:  
**obserwacja x** - wielowymiarowy wektor cech o wymiarze zgodnym z wymiarem danych.  
**mapowanie y** - dwu- lub trzy- wymiarowy wektor określający pozycje obserwacji na mapie.

1. Obliczamy odległosci pomiędzy obserwacjami poprzez wyliczenie odpowiednich prawdopodobieństw warunkowych.
Im bardziej obserwacje są podobne do siebie, tym większe prawdopodobieństwo. 

$$p_{j|i} = \frac{\exp{(-d(\boldsymbol{x}_i, \boldsymbol{x}_j) / (2 \sigma_i^2)})}{\sum_{i \neq k} \exp{(-d(\boldsymbol{x}_i, \boldsymbol{x}_k) / (2 \sigma_i^2)})}, \quad p_{i|i} = 0,$$

Powyższy wzorek określa podobieństwo dwóch obserwacji z wykorzystaniem rozkładu normalnego scentrowanego w $x_{i}$. $\sigma_{i}$ - odpowiednik odchylenia standardowego. Wyznaczanie $\sigma_{i}$ - odsyłam do pracy źródłowej - 	
Visualizing Data using t-SNE
L. van der Maaten, and G. Hinton. Journal of Machine Learning Research (2008)

Aby skorzystać z symetrii wprowadzamy:  

$$p_{ij} = \frac{p_{j|i} + p_{i|j}}{2N}.$$

W ten sposób otrzymujemy macierz podobieństwa **P** (ang. *similarity matrix*)

2. Następnie rozważamy macierz **Q** stanowiącą podobne odwzorowanie co powyżej, ale dla mapowań. Z przyczyn, które na razie pomińmy prawdopodobieństwa nie są liczone z wykorzystaniem rozkładu normalnego, ale t-Studenta o jednym stopniu swobody (rozkład Cauchy'ego):

$$q_{ij} = \frac{(1 + ||\boldsymbol{y}_i - \boldsymbol{y}_j||^2)^{-1}}{\sum_{k \neq l} (1 + ||\boldsymbol{y}_k - \boldsymbol{y}_l||^2)^{-1}},$$

3. Zaszliśmy daleko. Rozważamy teraz takie "umiejscowienie" obserwacji w przestrzeni o 2 lub 3 wymiarach, aby jak najbardziej zminimalizować różnicę pomiędzy dwiema macierzami (P i Q). Zmieniamy oczywiście macierz mapowań **Q**. Odpowiada to minimalizacji dywergencji Kullbacka-Leibera (https://pl.wikipedia.org/wiki/Dywergencja_Kullbacka-Leiblera, https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence):

$$KL(P|Q) = \sum_{i \neq j} p_{ij} \log \frac{p_{ij}}{q_{ij}}$$


## Przykład 1

<img src="fig/caltech101_tsne.jpg" alt="drawing" width="600"/>

## Przykład 2

1. Użyjemy zbioru :https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html#sklearn.datasets.load_digits

In [ ]:
from sklearn.manifold import TSNE
from sklearn.datasets import load_digits
from sklearn.preprocessing import scale

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

2. Zawiera on 1797 obrazków cyfr w formacie 8x8 pikseli.

In [ ]:
digits = load_digits()
digits.data.shape

3. Przykładowe obserwacje:

In [ ]:
nrows, ncols = 2, 5
plt.figure(figsize=(10,5))
plt.gray()
for i in range(ncols * nrows):
    ax = plt.subplot(nrows, ncols, i + 1)
    ax.matshow(digits.images[i,...])
    plt.xticks([]); plt.yticks([])
    plt.title(digits.target[i])
plt.show()

4. Uporządkowanie danych (tylko, aby pomóc nam w wizualizacji, algorytm tego nie wymaga)

In [ ]:
X = np.vstack([digits.data[digits.target==i]
               for i in range(10)])
y = np.hstack([digits.target[digits.target==i]
               for i in range(10)])

5. Wywołanie PCA i wizualizacja

In [ ]:
def scatter(x, colors):
    palette = np.array(sns.color_palette("hls", 10))

    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40,
                    c=palette[colors.astype(np.int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # add labels
    txts = []
    for i in range(10):
        # Position of each label.
        xtext, ytext = np.median(x[colors == i, :], axis=0)
        txt = ax.text(xtext, ytext, str(i), fontsize=24)
        txts.append(txt)

    return f, ax, sc, txts


In [ ]:
pca = PCA(n_components = 2)
digits_proj_pca = pca.fit_transform(X)
scatter(digits_proj_pca, y)
plt.show()

6. Wywołanie t-SNE

In [ ]:
%%time
random_state = 1500100900
tSNE = TSNE(random_state=random_state, verbose=1)
digits_proj = tSNE.fit_transform(X)

In [ ]:
scatter(digits_proj, y)
plt.show()

In [ ]:
tSNE.fit(X).get_params()

7. t-SNE bywa kosztowne obliczeniowo nawet dla niewielkich zbiorów danych. Aby skrócić czas obliczeń, jeśli liczba cech jest znacząca, można połączyć oba podejścia: PCA + tSNE.

In [ ]:
pca =  PCA().fit(X)

plt.figure(figsize=(9,6))
plt.plot(range(1, len(pca.explained_variance_ratio_)+1), np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

8. Spróbujmy z `n_components=40`

In [ ]:
np.cumsum(PCA(n_components=40).fit(X).explained_variance_ratio_)[-1]

In [ ]:
%%time
X_pca =  PCA(n_components=40).fit_transform(X)
random_state = 1500100900
digits_proj = TSNE(random_state=random_state).fit_transform(X_pca)

In [ ]:
scatter(digits_proj, y)
plt.show()

9. Dlaczego t?

W algorytmie SNE zarówno dla macierzy **P** (odwzorowuje odległości między obserwacjami) jak i **Q** (między mapowaniami) wykorzystuje się rozkład Gaussa. Okazuje się, że prowadzi to często do zagęszczenia mapowań. Obserwacje średnio odległe od siebie uzyskują bliskie sobie mapowania. (the crowding problem)

Problem ten można zniwelować poprzez wykorzystanie dla mapowań rozkładu t-Studenta z jednym stopniem swobody (rozkład Cauchy'ego), który pozwala na lepsze odwzorowanie tych odległości dzięki własności grubych ogonów. Prowadzi to do lepszego odseparowania danych w mapowaniu.

In [ ]:
z = np.linspace(0., 5., 1000)
gauss = np.exp(-z**2)
cauchy = 1/(1+z**2)
plt.plot(z, gauss, label='Gaussian distribution')
plt.plot(z, cauchy, label='Cauchy distribution')
plt.legend()
plt.show()

10. Uwagi końcowe o t-SNE:
 - służy do wizualizacji danych wielowymiarowych,
 - zwykle znajduje zastosowanie dla danych od 5 do 50 wymiarów, 
 - wykorzystuje rozkład t-Studenta zamiast rozkładu normalnego (SNE), aby przeciwdziałać zbyt małym odstępom pomiędzy średnio-odległymi obserwacjami,
 - często jest wykorzystywane wraz z PCA, aby zmiejszyć czas obliczeń. 

Źródła:
   -  https://github.com/oreillymedia/t-SNE-tutorial,
   -  http://jmlr.csail.mit.edu/papers/volume9/vandermaaten08a/vandermaaten08a.pdf,
   -  https://nbviewer.jupyter.org/urls/gist.githubusercontent.com/AlexanderFabisch/1a0c648de22eff4a2a3e/raw/59d5bc5ed8f8bfd9ff1f7faa749d1b095aa97d5a/t-SNE.ipynb